In [35]:
import pandas as pd
import numpy as np
import os, sys
sys.path.insert(0, os.path.abspath('../..'))
import source.data_handling.data_preparation as dp
import source.trading_strategies.trading_strategy as tsh

import numpy as np
import pandas as pd


In [36]:
pred_df = pd.read_csv("../../Data/ModelData/Classification/pred_df_loop_logit.csv", index_col=0, parse_dates=True)
bt = tsh.backtest_paper_regime_switch(
    pred_df,
    price_col="M1WO_O",
    regime_col="y_pred",
    ts_col="timestamp",
    bear_label=0,
)
summary = tsh.compare_regime_strategies(bt, periods_per_year=252, gamma=5.0, benchmark="BuyHoldEq")
print(summary)

             TotalReturn      CAGR    AnnVol  Sharpe(excess)  MaxDrawdown  \
Strategy                                                                    
Strategy        0.963304  0.042023  0.092020        0.493502    -0.246743   
BuyHoldEq       4.025733  0.103532  0.148473        0.738206    -0.340332   
BuyHoldRF       0.000000  0.000000  0.000000             NaN     0.000000   
Static50_50     1.346223  0.053413  0.074237        0.738206    -0.183277   

             AnnUtility  Δu vs BuyHoldEq  
Strategy                                  
Strategy       0.024243        -0.030250  
BuyHoldEq      0.054493         0.000000  
BuyHoldRF      0.000000        -0.054493  
Static50_50    0.041024        -0.013469  


/var/folders/q9/d545ygdx5rj6lg_00gpykppw0000gn/T/ipykernel_50231/2869902700.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pred_df = pd.read_csv("../../Data/ModelData/Classification/pred_df_loop_logit.csv", index_col=0, parse_dates=True)


In [37]:
pred_df

,timestamp,y_true,y_pred,pred_prob,M1WO_O
0,2010-01-01,0,0,0.286394,2796.04
1,2010-01-04,0,0,0.319418,2846.21
2,2010-01-05,0,0,0.186105,2854.58
3,2010-01-06,0,0,0.142301,2856.52
4,2010-01-07,0,0,0.124358,2855.09
...,...,...,...,...,...
4126,2025-10-27,0,0,0.075678,14141.13
4127,2025-10-28,0,0,0.057691,14164.97
4128,2025-10-29,0,0,0.066157,14161.08
4129,2025-10-30,0,0,0.094534,14022.56


In [38]:
df = pd.read_csv("../../Data/MSCI_World_Data.csv")

In [39]:
df[df["timestamp"] >= "2010-01-01"]

,timestamp,CL1,M1WO,SXXT,SPX,NKY,SPTR,EUR003M,FEDL01,GC1,V2X,MOVE,VIX,USYC2Y10,VXJ
2608,2010-01-01,79.36,2796.04,425.80,1115.10,10546.44,1837.499,0.700,0.05,1096.2,24.0577,107.50,21.68,269.734,22.19
2609,2010-01-04,81.51,2846.21,433.53,1132.98,10654.79,1867.063,0.700,0.12,1118.3,23.5136,106.40,20.04,274.962,20.71
2610,2010-01-05,81.77,2854.58,434.19,1136.52,10681.83,1872.896,0.699,0.12,1118.7,23.1342,102.50,19.35,275.081,20.60
2611,2010-01-06,83.18,2856.52,434.70,1137.14,10731.45,1874.729,0.694,0.12,1136.5,22.9753,106.90,19.16,282.744,19.94
2612,2010-01-07,82.66,2855.09,434.61,1141.70,10681.66,1882.344,0.692,0.10,1133.7,22.6947,104.40,19.06,279.751,20.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6734,2025-10-27,61.31,14141.13,1379.76,6875.16,50512.32,15250.144,2.084,4.12,4001.9,16.3429,66.87,15.79,48.888,30.75
6735,2025-10-28,60.15,14164.97,1379.64,6890.89,50219.18,15285.105,2.074,4.12,3966.2,16.4372,67.30,16.42,48.380,26.56
6736,2025-10-29,60.48,14161.08,1379.02,6890.59,51307.65,15284.535,2.066,4.12,3983.7,17.0345,65.75,16.92,47.567,27.96
6737,2025-10-30,60.57,14022.56,1377.22,6822.34,51325.61,15133.233,2.050,3.87,4015.9,16.8744,66.88,16.91,48.466,25.79
